In [2]:
import tensorflow as tf
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
import pandas as pd
import numpy as np

In [104]:
data = pd.read_csv('Advertising (2).csv')
data.columns
data.drop(['Unnamed: 0'], axis=1, inplace=True)
data.head()


,TV,Radio,Newspaper,Sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5
4,180.8,10.8,58.4,12.9


In [115]:
X_train, X_test, y_train, y_test = train_test_split(data[['TV','Radio','Newspaper']].values,data['Sales'].values)

In [116]:
X_train.shape

(150, 3)

In [117]:
ss = StandardScaler()
X_train = ss.fit_transform(X_train)
X_test = ss.fit_transform(X_test)

In [118]:
data['Sales'].values

array([ 22.1,  10.4,   9.3,  18.5,  12.9,   7.2,  11.8,  13.2,   4.8,
        10.6,   8.6,  17.4,   9.2,   9.7,  19. ,  22.4,  12.5,  24.4,
        11.3,  14.6,  18. ,  12.5,   5.6,  15.5,   9.7,  12. ,  15. ,
        15.9,  18.9,  10.5,  21.4,  11.9,   9.6,  17.4,   9.5,  12.8,
        25.4,  14.7,  10.1,  21.5,  16.6,  17.1,  20.7,  12.9,   8.5,
        14.9,  10.6,  23.2,  14.8,   9.7,  11.4,  10.7,  22.6,  21.2,
        20.2,  23.7,   5.5,  13.2,  23.8,  18.4,   8.1,  24.2,  15.7,
        14. ,  18. ,   9.3,   9.5,  13.4,  18.9,  22.3,  18.3,  12.4,
         8.8,  11. ,  17. ,   8.7,   6.9,  14.2,   5.3,  11. ,  11.8,
        12.3,  11.3,  13.6,  21.7,  15.2,  12. ,  16. ,  12.9,  16.7,
        11.2,   7.3,  19.4,  22.2,  11.5,  16.9,  11.7,  15.5,  25.4,
        17.2,  11.7,  23.8,  14.8,  14.7,  20.7,  19.2,   7.2,   8.7,
         5.3,  19.8,  13.4,  21.8,  14.1,  15.9,  14.6,  12.6,  12.2,
         9.4,  15.9,   6.6,  15.5,   7. ,  11.6,  15.2,  19.7,  10.6,
         6.6,   8.8,

In [119]:
y_train.shape

(150,)

In [120]:
tf.reset_default_graph()
X = tf.placeholder(dtype=tf.float32, shape=(None,3), name='X')
y=tf.placeholder(dtype=tf.float32, shape=(None), name='y')

h1=tf.layers.dense(X,3, name='hidden1',activation=tf.nn.relu)
h2=tf.layers.dense(h1,2, name='hidden2',activation=tf.nn.relu)
h3=tf.layers.dense(h2,6, name='hidden3',activation=tf.nn.relu)
y_hat = tf.layers.dense(h3, 1, name='y_hat', activation=None)

loss = tf.sqrt(tf.reduce_mean(tf.square(y_hat-y)))
gd = tf.train.AdamOptimizer(.01)
training_op = gd.minimize(loss)


In [129]:
kf = KFold(45)
batch_size_indices =[test for _, test in kf.split(X_train, y_train)]
init = tf.global_variables_initializer()
saver=tf.train.Saver()

In [130]:

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(200):
        for batch in batch_size_indices:
            X_batch = X_train[batch]
            y_batch = y_train[batch]
            sess.run(training_op, feed_dict={X: X_batch, y:y_batch})
        train_loss = sess.run(loss, feed_dict={X:X_train, y:y_train})
        test_loss= sess.run(loss, feed_dict={X:X_test,y:y_test})
        
        print("epoch", epoch, "train loss", train_loss,"test loss", test_loss)
        
    pred = sess.run(y_hat, feed_dict={X:X_test, y:y_test})
print(metrics.r2_score(y_test,pred))

epoch 0 train loss 11.5182 test loss 12.0879
epoch 1 train loss 6.12347 test loss 6.13257
epoch 2 train loss 5.40295 test loss 5.47792
epoch 3 train loss 5.37041 test loss 5.41567
epoch 4 train loss 5.40532 test loss 5.46165
epoch 5 train loss 5.47238 test loss 5.53451
epoch 6 train loss 5.50946 test loss 5.56228
epoch 7 train loss 5.52896 test loss 5.58172
epoch 8 train loss 5.5456 test loss 5.59599
epoch 9 train loss 5.5493 test loss 5.60242
epoch 10 train loss 5.55329 test loss 5.60708
epoch 11 train loss 5.55322 test loss 5.60809
epoch 12 train loss 5.55158 test loss 5.61285
epoch 13 train loss 5.55622 test loss 5.61841
epoch 14 train loss 5.5598 test loss 5.62447
epoch 15 train loss 5.56614 test loss 5.63042
epoch 16 train loss 5.56692 test loss 5.63441
epoch 17 train loss 5.57381 test loss 5.64261
epoch 18 train loss 5.57366 test loss 5.64697
epoch 19 train loss 5.5784 test loss 5.65102
epoch 20 train loss 5.57849 test loss 5.65308
epoch 21 train loss 5.58457 test loss 5.65818
ep

epoch 179 train loss 5.5618 test loss 5.69396
epoch 180 train loss 5.55576 test loss 5.69062
epoch 181 train loss 5.56412 test loss 5.70303
epoch 182 train loss 5.56685 test loss 5.6978
epoch 183 train loss 5.56232 test loss 5.69301
epoch 184 train loss 5.55622 test loss 5.69223
epoch 185 train loss 5.55903 test loss 5.69974
epoch 186 train loss 5.56771 test loss 5.70377
epoch 187 train loss 5.56357 test loss 5.69401
epoch 188 train loss 5.56321 test loss 5.69354
epoch 189 train loss 5.56275 test loss 5.69471
epoch 190 train loss 5.5555 test loss 5.69334
epoch 191 train loss 5.56096 test loss 5.69879
epoch 192 train loss 5.56613 test loss 5.6965
epoch 193 train loss 5.55714 test loss 5.6943
epoch 194 train loss 5.55746 test loss 5.70056
epoch 195 train loss 5.5665 test loss 5.70324
epoch 196 train loss 5.56411 test loss 5.69608
epoch 197 train loss 5.55898 test loss 5.70217
epoch 198 train loss 5.55953 test loss 5.69876
epoch 199 train loss 5.56166 test loss 5.69662
0.608456471834
